<a href="https://colab.research.google.com/github/ChrisM2309/LangchainTesting/blob/main/FinanceBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain

In [5]:
pip install langchain_community openai

In [3]:
!pip install openai

Haciendo los import y configurando la api key

In [78]:
import os
from openai import OpenAI as OpenAI

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI as OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

import json

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [11]:
#LLM TYPE
llm = OpenAI(temperature = 0.5)
llm_text = "Say hello world in python"
print(llm.predict(llm_text))

<ipython-input-11-602f8cb4f282>:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(llm_text))




print("Hello World")


In [14]:
#ChatBot Type
chat = ChatOpenAI(model="gpt-4o-mini")
#chat_text = "Dame las entidades y relaciones para una base de datos de una veterinaria" #Aqui van las preguntas
#messages = [HumanMessage(content=chat_text)]
#print(chat.predict_messages(messages).content)

<ipython-input-14-84c2753dc516>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4o-mini")


# CREAR TOOLS


In [126]:
# CHAINS
# Crear una cadena (Chain) para registrar transacciones
prompt_registrar = PromptTemplate(
    input_variables=["input"],
    template=""""
    Del input recibido, analiza con precaucion y extrae los parametros necesarios para hacer el registro:
    - Tipo (Ingreso o Gasto)
    - Monto (número)
    - Categoría (texto)
    - Descripción (texto, opcional, usa '' si no está presente)

    Texto: "{input}"

    Devuelve la respuesta en formato "(tipo) registrado con (monto) en (categoria) con (descripcion).".
    """
)

chain_registrar = LLMChain(llm=chat, prompt=prompt_registrar)

tool_registrar = Tool(
    name="registrar_transaccion",
    func=lambda x: chain_registrar.run(input=x),
    description="Sirve para registrar un ingreso o gasto con tipo (Ingreso/Gasto), monto (número), categoria (texto) y descripcion (texto) (opcional).")


# CALCULO DE FLUJO DE CAJA
prompt_flujo_caja = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para calcular el flujo de caja:
    - Ingresos (número)
    - Gastos (número)

    Texto: "{input}"

    Calcula el saldo (ingresos - gastos) y devuelve el resultado en formato:
    "🔹 Ingresos: $(ingresos) 🔹 Gastos: $(gastos)🔹 Saldo: $(saldo)"
    """
)
chain_flujo_caja = LLMChain(llm=llm, prompt=prompt_flujo_caja)

tool_flujo_caja = Tool(
    name="calcular_flujo_caja",
    func=lambda x: chain_flujo_caja.run(input=x),
    description="Calcula el flujo de caja a partir de ingresos y gastos."
)


# CONFIGURAR PRESUPUESTO
prompt_presupuesto = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para configurar un presupuesto:
    - Categoría (texto)
    - Monto (número)

    Texto: "{input}"

    Devuelve el resultado en formato: "✅ Presupuesto de ${monto} para {categoria} configurado."
    """
)

chain_presupuesto = LLMChain(llm=llm, prompt=prompt_presupuesto)

tool_presupuesto = Tool(
    name="configurar_presupuesto",
    func= lambda x: chain_presupuesto.run(input=x),
    description="Configura un presupuesto para una categoría."
)

# RESPONDER PREGUNTAS FINANCEIRAS
prompt_preguntas = PromptTemplate(
    input_variables=["input"],
    template="""
    Responde la siguiente pregunta financiera de manera clara y sencilla:
    "{input}"
    Usa toda la informacion disponible para responder la pregunta. Es crucial que consultes la memoria para obtener datos relevantes.
    """
)
chain_preguntas = LLMChain(llm=llm, prompt=prompt_preguntas)

tool_preguntas = Tool(
    name="responder_pregunta",
    func=lambda x: chain_preguntas.run(input=x),
    description="Responde preguntas financieras con la informacion financieras generales o de la informacion del usuario y la memoria"
)

# CONFIGURAR AL AGENTE

In [119]:
memory = ConversationBufferMemory(memory_key="chat_history")

# Inicializar el agente
# Lista de herramientas
tools = [tool_registrar,tool_flujo_caja, tool_presupuesto, tool_preguntas]

# Inicializar el agente
agente = initialize_agent(
    tools = tools,
    llm = chat,
    agent="conversational-react-description",
    verbose=True,
    max_iterations = 3,
    memory = memory
    )

TESTEO DEL AGENTE

In [127]:
print(agente.run("Registra un gasto de $150 en insumos"))
print(agente.run("Calcula el flujo de caja con ingresos de $500 y gastos de $300."))
print(agente.run("Registra un gasto de $200 en publiicidad."))
print(agente.run("Cuanto he gastado en total?"))
print(agente.run("¿Qué es el flujo de caja?"))




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: registrar_transaccion
Action Input: {"tipo": "Gasto", "monto": 150, "categoria": "insumos", "descripcion": "Gasto en insumos"}
Observation: Gasto registrado con 150 en insumos con Gasto en insumos.
Thought:Do I need to use a tool? No
AI: El gasto de $150 en insumos ha sido registrado exitosamente. Si necesitas ayuda con algo más, no dudes en decírmelo.

> Finished chain.
El gasto de $150 en insumos ha sido registrado exitosamente. Si necesitas ayuda con algo más, no dudes en decírmelo.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: calcular_flujo_caja
Action Input: {"ingresos": 500, "gastos": 300}
Observation: 
    🔹 Ingresos: $500 🔹 Gastos: $300 🔹 Saldo: $200
Thought:Do I need to use a tool? No  
AI: El flujo de caja ha sido calculado. Tienes ingresos de $500 y gastos de $300, lo que resulta en un saldo de $200. Si necesitas más ayuda o información, házm